<a href="https://colab.research.google.com/github/lxbuzz/DataAnalysis/blob/main/seasonality_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You are the deputy director of the trading company "****", which sells fruit and vegetable products.
     The activity of the enterprise is significantly influenced by the factor of seasonality and the indicator of trade has pronounced seasonal fluctuations, which largely depends on the organization of the process of customer service. By 2021, the total annual turnover is expected in the amount of 2500 thousand UAH.
Seasonal fluctuations of turnover of the trading company "****"
thousand UAH
Years and Indicators

Turnover for the month
Total for the year





















































































 Forecast for 2021
























2500




In [21]:
import pandas as pd

# Load the data from the CSV file
#df = pd.read_csv("sales data - pls assis with forecast (see picture attached for....csv")

#!pip install gspread==5.7.2 oauth2client==4.1.3

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Open the Google Sheet by its title
sheet = gc.open("sales data").sheet1  # Replace "Your Google Sheet Title" with the actual title

# Read the data into a Pandas DataFrame
data = sheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])  # Assuming the first row contains column headers







# Convert relevant columns to numeric
for col in ['2018', '2019', '2020']:
    df[col] = pd.to_numeric(df[col])

df = df.drop(columns=['Sum18_20','Tot','Avg'])

# Set the 'Month' column as the index
df.set_index('Month', inplace=True)
df = df.drop('Total', axis=0)




# Calculate total turnover for each month
df['Total'] = df.sum(axis=1)

# Calculate average turnover per month (A_month)
df['A_month'] = df['Total'] / 3
print(df.head(15))
print(df.info())

# Calculate average turnover per year (A_year)
A_year = df.sum(axis=0).drop(['Total','A_month']).sum() / 3

print(A_year)

# Calculate seasonality factor (S_month)
df['S_month'] = df['A_month'] / (A_year / 12)

# Forecast for 2021 (F_year = 2500)
F_year = 2500
df['F_month'] = (F_year / 12) * df['S_month']

# Print the forecast for 2021
print(df['F_month'].round(2))

# Check calculations
print("\nSum of Seasonality Factors:", df['S_month'].sum().round(2))
print("Sum of Monthly Forecasts:", df['F_month'].sum().round(2))

print(df.head(15))

            2018   2019   2020  Total     A_month
Month                                            
January     70.0   76.0   73.0  219.0   73.000000
February    79.0   90.0   84.0  253.0   84.333333
March       90.0  107.0  102.0  299.0   99.666667
April      108.0  134.0  136.0  378.0  126.000000
May        136.0  149.0  154.0  439.0  146.333333
June       160.0  192.0  190.0  542.0  180.666667
July       197.0  220.0  214.0  631.0  210.333333
August     208.0  233.0  238.0  679.0  226.333333
September  182.0  212.0  216.0  610.0  203.333333
October    150.0  190.0  171.0  511.0  170.333333
November   121.0  111.0  124.0  356.0  118.666667
December    86.0   93.0   95.0  274.0   91.333333
             NaN    NaN    NaN    0.0    0.000000
<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, January to 
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   2018     12 non-null     float64
 1   2019     12 non-null     float

This code does the following:
 accurate forecast for 2021 based on the data

In [28]:


# Export the DataFrame to a new sheet named 'Forecast'
try:
    # Try to find the existing sheet
    worksheet = gc.open("sales data").worksheet("Forecast")
except gspread.exceptions.WorksheetNotFound:
    # If not found, create a new sheet
    worksheet = gc.open("sales data").add_worksheet(title="Forecast", rows="100", cols="20")

# Reset the index to include month names as a column
df_export = df.reset_index()

# Convert problematic float values to strings before exporting
df_export = df_export.astype(object).where(pd.notnull(df_export), None)

worksheet.update([df_export.columns.values.tolist()] + df_export.values.tolist())

{'spreadsheetId': '1RUVIZmt59niyDP3Hm0bffvf7h8a2TOxpMYse-QolGN4',
 'updatedRange': 'Forecast!A1:H14',
 'updatedRows': 14,
 'updatedColumns': 8,
 'updatedCells': 109}